In [1]:
from openai import OpenAI
import os
from enum import Enum
import jinja2

In [2]:
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key="voc-1142391097160736424292768839a5d4b7a92.77641675")

In [3]:
MODEL = "gpt-4o-mini"
def get_completion(messages=None, system_prompt=None, user_prompt=None, model=MODEL):
    messages = list(messages or [])
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    if user_prompt:
        messages.append({"role": "user", "content": user_prompt})
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
    )
    return response.choices[0].message.content

In [4]:
# Define sample recipes
sample_recipes = [
    {
        "name": "Classic Spaghetti Bolognese",
        "ingredients": [
            "1 lb ground beef",
            "1 onion, finely chopped",
            "2 garlic cloves, minced",
            "1 carrot, finely diced",
            "1 celery stalk, finely diced",
            "1 can (14 oz) crushed tomatoes",
            "2 tbsp tomato paste",
            "1 cup beef broth",
            "1 tsp dried oregano",
            "1 bay leaf",
            "1 lb spaghetti",
            "2 tbsp olive oil",
            "Salt and pepper to taste",
            "Grated Parmesan cheese for serving",
        ],
        "instructions": [
            "Heat olive oil in a large pot over medium heat.",
            "Add onion, garlic, carrot, and celery. Cook until softened, about 5 minutes.",
            "Add ground beef and cook until browned, breaking it up as it cooks.",
            "Stir in tomato paste and cook for 1 minute.",
            "Add crushed tomatoes, beef broth, oregano, bay leaf, salt, and pepper.",
            "Bring to a simmer, then reduce heat to low and cook for 1-2 hours.",
            "Cook spaghetti according to package instructions until al dente.",
            "Drain pasta and serve topped with the Bolognese sauce.",
            "Sprinkle with grated Parmesan cheese.",
        ],
    },
    {
        "name": "Vegetable Stir Fry",
        "ingredients": [
            "2 cups mixed vegetables (bell peppers, broccoli, carrots, snap peas)",
            "1 block firm tofu, cubed",
            "2 tbsp vegetable oil",
            "2 cloves garlic, minced",
            "1 tsp ginger, grated",
            "3 tbsp soy sauce",
            "1 tbsp rice vinegar",
            "1 tsp sesame oil",
            "1 tsp cornstarch",
            "2 green onions, sliced",
            "Sesame seeds for garnish",
            "Cooked rice for serving",
        ],
        "instructions": [
            "Press tofu to remove excess water, then cut into cubes.",
            "Mix soy sauce, rice vinegar, sesame oil, and cornstarch in a small bowl.",
            "Heat vegetable oil in a wok or large skillet over high heat.",
            "Add tofu and cook until golden, about 3-4 minutes. Remove and set aside.",
            "Add garlic and ginger to the wok and stir for 30 seconds.",
            "Add vegetables and stir-fry for 4-5 minutes until crisp-tender.",
            "Return tofu to the wok, add sauce mixture, and cook for 1-2 minutes until sauce thickens.",
            "Garnish with green onions and sesame seeds.",
            "Serve over rice.",
        ],
    },
    {
        "name": "Chocolate Chip Cookies",
        "ingredients": [
            "2 1/4 cups all-purpose flour",
            "1 tsp baking soda",
            "1 tsp salt",
            "1 cup (2 sticks) butter, softened",
            "3/4 cup granulated sugar",
            "3/4 cup packed brown sugar",
            "2 large eggs",
            "2 tsp vanilla extract",
            "2 cups semi-sweet chocolate chips",
            "1 cup chopped nuts (optional)",
        ],
        "instructions": [
            "Preheat oven to 375°F (190°C).",
            "Combine flour, baking soda, and salt in a small bowl.",
            "Beat butter, granulated sugar, and brown sugar in a large mixer bowl.",
            "Add eggs one at a time, beating well after each addition.",
            "Beat in vanilla extract.",
            "Gradually beat in flour mixture.",
            "Stir in chocolate chips and nuts if using.",
            "Drop by rounded tablespoon onto ungreased baking sheets.",
            "Bake for 9 to 11 minutes or until golden brown.",
            "Cool on baking sheets for 2 minutes; remove to wire racks to cool completely.",
        ],
    },
]

# Define common dietary restrictions
dietary_restrictions = [
    "vegetarian",
    "vegan",
    "gluten-free",
    "dairy-free",
    "nut-free",
    "egg-free",
    "low-sodium",
    "keto",
    "paleo",
    "kosher",
]

In [6]:
initial_prompt = """
Analyze the following recipe and determine whether it satisfies each dietary restriction in the list.
For each restriction, classify it as "satisfied", "not satisfied", or "undeterminable" based on the recipe information.

Recipe: {{ recipe_name }}

Ingredients:
{{ recipe_ingredients }}

Instructions:
{{ recipe_instructions }}

Dietary Restrictions to Check:
{{ dietary_restrictions }}

Please provide your response in JSON format.
"""

def format_prompt(recipe, prompt):
    ingredients_str = "\n".join(
        ["- " + ingredient for ingredient in recipe["ingredients"]]
    )
    instructions_str = "\n".join(
        [
            f"{i + 1}. {instruction}"
            for i, instruction in enumerate(recipe["instructions"])
        ]
    )
    restrictions_str = ", ".join(dietary_restrictions)

    return jinja2.Template(prompt).render(
        recipe_name=recipe["name"],
        recipe_ingredients=ingredients_str,
        recipe_instructions=instructions_str,
        dietary_restrictions=restrictions_str,
    )


In [7]:
# test
test_prompt = format_prompt(sample_recipes[0], initial_prompt)
test_response = get_completion(user_prompt=test_prompt)
print(test_response)

```json
{
  "vegetarian": "not satisfied",
  "vegan": "not satisfied",
  "gluten-free": "not satisfied",
  "dairy-free": "not satisfied",
  "nut-free": "satisfied",
  "egg-free": "satisfied",
  "low-sodium": "undeterminable",
  "keto": "not satisfied",
  "paleo": "not satisfied",
  "kosher": "undeterminable"
}
```


In [8]:

refined_prompt = """
You are a dietary consultant specializing in food allergies and dietary restrictions.

Your task is to analyze the following recipe and determine whether it satisfies each dietary restriction in the list.
For each restriction, classify it as "satisfied," "not satisfied," or "undeterminable" based on the recipe information.

Important context and definitions for dietary restrictions:
- Vegetarian: No meat, poultry, fish, or seafood. May include eggs and dairy.
- Vegan: No animal products whatsoever, including meat, dairy, eggs, honey.
- Gluten-free: No wheat, barley, rye, or derivatives. Note that regular all-purpose flour contains gluten.
- Dairy-free: No milk, cheese, butter, cream, or other dairy products.
- Nut-free: No tree nuts or peanuts.
- Egg-free: No eggs or products containing eggs.
- Low-sodium: Limited salt and naturally high-sodium ingredients.
- Keto: Very low carbohydrate, high fat, moderate protein.
- Paleo: No grains, legumes, dairy, refined sugar, or processed foods.
- Kosher: Follows Jewish dietary laws (no pork, shellfish, mixing meat and dairy, etc.).

Guidelines for your analysis:
- Mark a restriction as "satisfied" only if you are certain the recipe meets it.
- Mark a restriction as "not satisfied" if any ingredient clearly violates it.
- Mark a restriction as "undeterminable" if you lack sufficient information (e.g., exact type of broth, potential cross-contamination).
- For each classification, briefly explain your reasoning and identify specific ingredients that affect your decision.

Recipe: {{ recipe_name }}

Ingredients:
{{ recipe_ingredients }}

Instructions:
{{ recipe_instructions }}

Dietary Restrictions to Check:
{{ dietary_restrictions }}

Please format your response as a JSON object where:
- Each key is the name of a dietary restriction
- Each value is an object with properties:
  - "classification": "satisfied", "not satisfied", or "undeterminable"
  - "explanation": brief reasoning for your classification
  - "critical_ingredients": array of ingredients that determined your classification
"""

In [11]:
test_prompt_2 = format_prompt(sample_recipes[0], refined_prompt)
test_response_2 = get_completion(user_prompt=test_prompt_2)
print(f"Iteration 1 response for Spaghetti Bolognese:\n{test_response_2}\n")

Iteration 1 response for Spaghetti Bolognese:
```json
{
  "vegetarian": {
    "classification": "not satisfied",
    "explanation": "The recipe contains ground beef, which is meat.",
    "critical_ingredients": ["ground beef"]
  },
  "vegan": {
    "classification": "not satisfied",
    "explanation": "The recipe contains ground beef and grated Parmesan cheese, both of which are animal products.",
    "critical_ingredients": ["ground beef", "grated Parmesan cheese"]
  },
  "gluten-free": {
    "classification": "not satisfied",
    "explanation": "The recipe includes spaghetti, which typically contains wheat and therefore gluten.",
    "critical_ingredients": ["spaghetti"]
  },
  "dairy-free": {
    "classification": "not satisfied",
    "explanation": "The recipe includes grated Parmesan cheese, a dairy product.",
    "critical_ingredients": ["grated Parmesan cheese"]
  },
  "nut-free": {
    "classification": "satisfied",
    "explanation": "The recipe does not include any tree nuts 